In [1]:
from __future__ import print_function
import torch  #importing the torch library
import torch.nn as nn     #importing the neural network module
import torch.nn.functional as F      # importing 
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):         #base class for all the nn modules
    def __init__(self):       #initializing these parameters when the class object is created.
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1,16,3),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.25),

            nn.Conv2d(16,16,3),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.25),

            nn.Conv2d(16,16,3),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Dropout(0.25)
            # minimum 3 conv layers to achive 7*7 receptive field so that most of the edges and patterns are easily captured.
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(16,32,1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.25),

            nn.Conv2d(32,32,kernel_size=3),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout(0.25),

            nn.Conv2d(32,16,kernel_size=3),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(0.25)
            # stopping at 7*7 so that we can go into FC layers.
        )

        self.fc1 = nn.Sequential(
            nn.AvgPool2d(7)
            # GAP layer bafore FC dense layer
        )

        self.fc2 = nn.Sequential(
            # nn.Linear(64,32),
            # nn.ReLU(),
            # nn.BatchNorm1d(32),
            # nn.Dropout(0.25),
            # nn.Linear(32,16),
            # nn.ReLU(),
            # nn.Dropout(0.25),
            nn.Linear(16,10)
        )


    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.fc1(x)
        x = x.view(-1,16)
        x = self.fc2(x)
        return F.log_softmax(x)

In [3]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
           Dropout-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           2,320
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
           Dropout-8           [-1, 16, 24, 24]               0
            Conv2d-9           [-1, 16, 22, 22]           2,320
             ReLU-10           [-1, 16, 22, 22]               0
      BatchNorm2d-11           [-1, 16, 22, 22]              32
        MaxPool2d-12           [-1, 16, 11, 11]               0
          Dropout-13           [-1, 16, 11, 11]               0
           Conv2d-14           [-1, 32,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:62: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:

torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'epoch={epoch} loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0723, Accuracy: 9794/10000 (97.94%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0544, Accuracy: 9842/10000 (98.42%)



  0%|          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0201, Accuracy: 9936/10000 (99.36%)



  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0217, Accuracy: 9927/10000 (99.27%)



epoch=19 loss=0.00453563779592514 batch_id=937: 100%|██████████| 938/938 [00:18<00:00, 51.82it/s]



Test set: Average loss: 0.0201, Accuracy: 9934/10000 (99.34%)



#achived 99.42% accracy in epoch no 15 :) 